In [41]:
from matplotlib import pyplot as plt
import numpy as np
import mapper
import tensorflow as tf
from itertools import tee
import graphviz
from load_data import load_data
%matplotlib inline

In [125]:
sections, groups = load_data("csolid_cold.txt")
energies = np.array([sec.potential_energy for sec in sections])

In [126]:
for idx in range(200, 230):
    sect = groups[idx]
    pts = sect[:, :3]
    velocity = sect[:, 3:]
    speed = (velocity**2).sum(axis=-1).reshape(-1, 1)


    cov = mapper.cover.cube_cover_primitive()
    flt = velocity # mapper.filters.eccentricity(pts)
    with Capturing() as output:
        mappered = mapper.mapper(pts, flt, cov, mapper.cutoff.histogram(19))

    do_mat(mappered.simplices[1], "csolid_cold{}.png".format(idx))
# g = graphviz.Graph()
# for (a,) in mappered.simplices.simplices[0].keys():
#     g.node(str(a))

# for ((a, b), weight) in mappered.simplices.simplices[1].items():
#     g.edge(str(a), str(b), weight=str(weight))
# g

In [6]:
# http://stackoverflow.com/questions/16571150/how-to-capture-stdout-output-from-a-python-function-call

from io import StringIO
import sys

class Capturing(list):
    def __enter__(self):
        self._stdout = sys.stdout
        sys.stdout = self._stringio = StringIO()
        return self
    def __exit__(self, *args):
        self.extend(self._stringio.getvalue().splitlines())
        del self._stringio    # free up some memory
        sys.stdout = self._stdout

In [121]:
from PIL import Image
sess = tf.Session()

def do_mat(sim, name):
    items1, items2 = tee(sim.items())
    keys = [k for k,v in items1]
    values = [v for k,v in items2]
    sparse = tf.SparseTensor(indices=keys, values=values, dense_shape=[1500] * len(keys[0]))
    img = sess.run(tf.sparse_tensor_to_dense(sparse))
    img = ((img - 1) * -255).astype("uint8")
    i = Image.fromarray(img)
    i.save(name)